# Data Frames

a DataFrame is a popular data structure (in many languages) for storing data in tabular form (rows, columns)
the Julia package DataFrames.jl has many functions that operate on DataFrames that allow us to quickly query and manipulate data
it can handle missing values efficiently
the Julia package CSV.jl allows us to quickly and flexibly read/write CSV (= comma-separated value) files from/to DataFrames
You need to install these packages:

```bash
using Pkg
Pkg.add("DataFrames")
Pkg.add("CSV")
```

In [1]:
# always list packages you use at the top
using DataFrames # load into namespace all of the functions, data types, variables from the `DataFrames` package
using CSV
using Printf # for print formating
using Statistics # for `mean`

## Construct a `DataFrame`
This construction approach is an array-styel that has the order memorized.

In [12]:
city = ["Corvallis", "Portland", "Eugene"]
population = [57961, 647805, 168916]

df_cities = DataFrame(city=city, population=population)

Row,city,population
,String,Int64
1,Corvallis,57961
2,Portland,647805
3,Eugene,168916


## add rows to the `Data Frame`

In [13]:
push!(df_cities, ["Bend", 94520]) #If you do not enter the `!` it will raise an error

Row,city,population
,String,Int64
1,Corvallis,57961
2,Portland,647805
3,Eugene,168916
4,Bend,94520


###  Dictionary style
Order is not memorized

In [14]:
push!(df_cities, Dict(:population => 122324, :city => "Berkeley"))
df_cities

Row,city,population
,String,Int64
1,Corvallis,57961
2,Portland,647805
3,Eugene,168916
4,Bend,94520
5,Berkeley,122324


## add columns to the `DataFrame`

In [17]:
# one way, inserts new column as the last column
df_cities[!, :state] = vcat(["Oregon" for i = 1:4], ["California"])
df_cities

Row,city,population,state
,String,Int64,String
1,Corvallis,57961,Oregon
2,Portland,647805,Oregon
3,Eugene,168916,Oregon
4,Bend,94520,Oregon
5,Berkeley,122324,California


In [21]:
# another way, allows you to insert a column at specified location (e.g., 2)
insertcols!(df_cities, 2, :rainfall=>[51.0, 43.0, 47.0, 12.0, 25.0])
df_cities

Row,city,rainfall,population,state
,String,Float64,Int64,String
1,Corvallis,51.0,57961,Oregon
2,Portland,43.0,647805,Oregon
3,Eugene,47.0,168916,Oregon
4,Bend,12.0,94520,Oregon
5,Berkeley,25.0,122324,California


### how many (rows,columns)?

In [22]:
size(df_cities) # size(df_cities)[1] = # rows, size(df_cities)[2] = # cols

(5, 4)

### What are the column names? 
note that the column names are always symbols! if you have a fancy String you want to convert into a Symbol, simply e.g. Symbol("avg salary (USD)") will convert the String "avg salary (USD)" to a Symbol.

In [23]:
names(df_cities)

4-element Vector{String}:
 "city"
 "rainfall"
 "population"
 "state"

In [27]:
for row in eachrow(df_cities)
    @printf("population of %s, %s is %d\n", row[:city], row[:state], row[:population])
end

population of Corvallis, Oregon is 57961
population of Portland, Oregon is 647805
population of Eugene, Oregon is 168916
population of Bend, Oregon is 94520
population of Berkeley, California is 122324


## Retreive a column
The columns are arrays!

In [32]:
# (1) treating the DataFrame like a multi-dimensional array
df_cities[:, :population] # : or ! means all the rows

5-element Vector{Int64}:
  57961
 647805
 168916
  94520
 122324

In [29]:
# (2) treating the DataFrame like a `Struct`
df_cities.population

5-element Vector{Int64}:
  57961
 647805
 168916
  94520
 122324

## Retrieve a row

In [34]:
df_cities[2,:] # Pull all columns from the second row

Row,city,rainfall,population,state
,String,Float64,Int64,String
2,Portland,43.0,647805,Oregon


## Retrieve an entry

In [35]:
df_cities[2, :population] # population of Portland

647805

## Retrieve a combination of rows/columns

like an array we can retreive, the first 3 rows and only the `population` and `city` columns

In [36]:
df_cities[1:3, [:population, :city]]

Row,population,city
,Int64,String
1,57961,Corvallis
2,647805,Portland
3,168916,Eugene


In [37]:
df_cities

Row,city,rainfall,population,state
,String,Float64,Int64,String
1,Corvallis,51.0,57961,Oregon
2,Portland,43.0,647805,Oregon
3,Eugene,47.0,168916,Oregon
4,Bend,12.0,94520,Oregon
5,Berkeley,25.0,122324,California


## Querying a dataframe
Using the filter funtion:
* the first argument is a function that operates on a row of the DataFrame (treating the row as a Dict) and returns true if we want to keep that row and false if we want to throw it away
* the second argument is the dataframe
* there is also a filter! function that will remove these rows from the dataframe instead of returning a copy with the relevant rows removed

In [38]:
filter(row -> row[:city] == "Corvallis", df_cities)

Row,city,rainfall,population,state
,String,Float64,Int64,String
1,Corvallis,51.0,57961,Oregon


In [43]:
df_cities[df_cities[:, :population] .< 500000, :] # all the rows where the population is less than 500,000

Row,city,rainfall,population,state
,String,Float64,Int64,String
1,Corvallis,51.0,57961,Oregon
2,Eugene,47.0,168916,Oregon
3,Bend,12.0,94520,Oregon
4,Berkeley,25.0,122324,California


In [44]:
filter(row -> row[:state] == "Oregon", df_cities) # all the rows where cities are in the state of Oregon

Row,city,rainfall,population,state
,String,Float64,Int64,String
1,Corvallis,51.0,57961,Oregon
2,Portland,43.0,647805,Oregon
3,Eugene,47.0,168916,Oregon
4,Bend,12.0,94520,Oregon


## Sorting a dataframe
e.g. permute rows so that cities are listed by :population in reverse (rev) order

In [45]:
sort!(df_cities, :population, rev=true) # ! means the input dataframe was modified. now the rows are re-ordered

Row,city,rainfall,population,state
,String,Float64,Int64,String
1,Portland,43.0,647805,Oregon
2,Eugene,47.0,168916,Oregon
3,Berkeley,25.0,122324,California
4,Bend,12.0,94520,Oregon
5,Corvallis,51.0,57961,Oregon
